In [50]:
# imports

import pandas as pd
import numpy as np
import tensorflow as tf

import keras
from keras import layers, optimizers, losses, metrics

In [51]:
import os

data_path = "./data/transformed/"

if any(["COLAB" in x for x in list(os.environ.keys())]):
    from google.colab import drive
    drive.mount('/content/drive/')
    data_path = r"/content/drive/MyDrive/Colab Notebooks/digit-recognizer/data/input/"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [52]:
!ls -R {data_path.replace(" ", "\\ ")}

'/content/drive/MyDrive/Colab Notebooks/digit-recognizer/data/input/':
digit-recognizer.zip  sample_submission.csv  test.csv  train.csv


In [59]:
import tensorflow as tf
import pandas as pd
from dataclasses import dataclass

@dataclass
class DigitTensor:
    X: tf.Tensor
    y: tf.Tensor


class Transform:
    def __init__(self, data_path, classes):
        self._data_path = data_path
        self.classes = classes
        self.train_df = pd.read_csv(data_path + "train.csv")
        self.test_df = pd.read_csv(data_path + "test.csv")

    def transform(self, train = True):
        if train:
            X, y = self.train_df.iloc[:, 1:].to_numpy(), self.train_df.iloc[:, 0].to_numpy()

            # onehot encode labels
            y = self.transform_y(y)
        else:
            X, y = self.test_df.iloc[:, 0:].to_numpy(), None

        return self.transform_X(X), y

    def transform_X(self, X):
        # Reshape Row Pixels to Image (height x width)
        X = X.reshape(-1, 28, 28)

        # Rescale Image by padding (32, 32)
        X = np.pad(X, 2, mode = 'edge')[2:-2]

        # Reshape to include channel (height x width x channels)
        X = X.reshape(-1, 32, 32, 1)

        # Include RGB channel (height x width x 3)
        X = np.repeat(X, 3, axis = -1)

        # Normalize
        X = X / 255.

        return tf.constant(X)

    def transform_y(self, y):
        return tf.one_hot(y, depth = self.classes)

    def train_test_split(self, X, y, train_ratio = 0.8):
        num = X.shape[0]

        # generate indicies
        indicies = np.arange(num, dtype = np.int32)

        #  shuffle
        np.random.shuffle(indicies)

        # get indicies
        stop_index = int(train_ratio * num)
        train_indicies, test_indicies = indicies[:stop_index].tolist(), indicies[stop_index:].tolist()

        X_train, y_train = tf.gather(X, train_indicies), tf.gather(y, train_indicies)
        X_test, y_test = tf.gather(X, test_indicies), tf.gather(y, test_indicies)

        return DigitTensor(X_train, y_train), DigitTensor(X_test, y_test)


In [60]:
digit_input_tranformer = Transform(data_path = data_path, classes = 10)

X, y = digit_input_tranformer.transform()
X_test_unseen, y_test_unseen = digit_input_tranformer.transform(train = False)

train_tensor, test_tensor = digit_input_tranformer.train_test_split(X, y)

In [61]:
from tensorflow.keras.applications.resnet import ResNet50

In [62]:
resnet = ResNet50(include_top = False, input_shape = train_tensor.X.shape[1:], pooling = 'avg')

resnet.trainable = True

# resnet.summary()

classifier = keras.Sequential(
    [
        resnet,
        layers.Dense(128, activation = 'sigmoid'),
        layers.Dense(10, activation = 'softmax')
    ]
)


classifier.compile(
    optimizer = optimizers.Adam(learning_rate = 1e-3, decay = 1e-3),
    loss = losses.CategoricalCrossentropy(),
    metrics = [metrics.CategoricalAccuracy()]
)

classifier.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_10 (Dense)            (None, 128)               262272    
                                                                 
 dense_11 (Dense)            (None, 10)                1290      
                                                                 
Total params: 23,851,274
Trainable params: 23,798,154
Non-trainable params: 53,120
_________________________________________________________________


In [63]:
history = classifier.fit(
    train_tensor.X,
    train_tensor.y,
    epochs = 30,
    batch_size = 32,
    validation_data = (test_tensor.X, test_tensor.y))

Epoch 1/30
1050/1050 [==============================] - 49s 40ms/step - loss: 0.2238 - categorical_accuracy: 0.9433 - val_loss: 0.1113 - val_categorical_accuracy: 0.9699
Epoch 2/30
1050/1050 [==============================] - 40s 38ms/step - loss: 0.0710 - categorical_accuracy: 0.9823 - val_loss: 0.0489 - val_categorical_accuracy: 0.9874
Epoch 3/30
1050/1050 [==============================] - 42s 40ms/step - loss: 0.0461 - categorical_accuracy: 0.9888 - val_loss: 0.0362 - val_categorical_accuracy: 0.9899
Epoch 4/30
1050/1050 [==============================] - 42s 40ms/step - loss: 0.0302 - categorical_accuracy: 0.9925 - val_loss: 0.0339 - val_categorical_accuracy: 0.9911
Epoch 5/30
1050/1050 [==============================] - 40s 38ms/step - loss: 0.0231 - categorical_accuracy: 0.9942 - val_loss: 0.0346 - val_categorical_accuracy: 0.9905
Epoch 6/30
1050/1050 [==============================] - 40s 38ms/step - loss: 0.0160 - categorical_accuracy: 0.9959 - val_loss: 0.0400 - val_categoric

In [64]:
from datetime import datetime


# Predict on test unseen
y_unseen_predicted = classifier.predict(X_test_unseen)

# Transform labels
y_unseen_predicted = tf.argmax(y_unseen_predicted, axis = 1).numpy()

now = datetime.now().strftime("%Y%m%d_%H%M%S")

output = pd.read_csv(data_path + "sample_submission.csv", index_col=0)
output['Label'] = y_unseen_predicted

output.to_csv(f"{data_path}/../output/digit_recognizer_{now}.csv", index=True)

875/875 [==============================] - 9s 10ms/step
